In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import re
import tensorflow as tf
from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as layers
from tensorflow.keras import Input
from absl import logging
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import urllib
from custom_gen import DataGenerator
import glob
import json

In [6]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
embedding_model = hub.load(module_url)
def embed(input):
  return embedding_model(input)
original_data = pd.read_csv("data/mbti_1.csv")

In [7]:
imgs = {i:[] for i in range(original_data.shape[0])}

for i, person in enumerate(original_data['posts']):
    urls_person = [re.findall("(?P<url>https?://[^\s]+)", post) for post in person.split('|||')]
    original_data['posts'].iloc[i] = [re.sub("(?P<url>https?://[^\s]+)", '', post) for post in person.split('|||')]
    urls_clean = []
    for url in urls_person: 
        if url:
            urls_clean.append(url[0])
    for url in urls_clean:
        if (".jpg" in url) or (".png" in url):
            imgs[i].append(url)

In [9]:
total = []
for i in range(original_data.shape[0]):
    if len(imgs[i])!=0:
        total.append(i)
print(len(total))

2423


In [4]:
original_data = pd.read_csv("data/mbti_1.csv")
def remove_links_and_embed(posts):
    return embed([re.sub("(?P<url>https?://[^\s]+)", '', post) for post in posts.split('|||')]).numpy()

original_data['posts'] = original_data['posts'].apply(remove_links_and_embed)

code = {tp:i for i,tp in enumerate(pd.unique(original_data['type']))}
rev_code  = {i:tp for i,tp in enumerate(pd.unique(original_data['type']))}

In [7]:
original_data['posts'].head()[0]

array([[-0.03565682, -0.03319573,  0.0704743 , ..., -0.0636081 ,
        -0.02079295, -0.02250968],
       [-0.03565682, -0.03319573,  0.0704743 , ..., -0.0636081 ,
        -0.02079295, -0.02250968],
       [-0.05830672,  0.00682049, -0.05866492, ...,  0.01276692,
        -0.00112303, -0.06254279],
       ...,
       [ 0.04250416, -0.08287645, -0.0163145 , ..., -0.01025333,
        -0.0569249 ,  0.01330812],
       [ 0.05174695,  0.00672889, -0.06231392, ...,  0.06926323,
         0.04425861, -0.07807797],
       [ 0.03404305, -0.07008429, -0.03211437, ..., -0.00176   ,
         0.03975417, -0.04023048]], dtype=float32)

In [5]:
original_data

,type,posts
0,INFJ,"((tf.Tensor(-0.035656825, shape=(), dtype=floa..."
1,ENTP,"((tf.Tensor(0.006778221, shape=(), dtype=float..."
2,INTP,"((tf.Tensor(-0.009484898, shape=(), dtype=floa..."
3,INTJ,"((tf.Tensor(-0.009333394, shape=(), dtype=floa..."
4,ENTJ,"((tf.Tensor(-0.038833104, shape=(), dtype=floa..."


In [ ]:
vision_model = Sequential()
vision_model.add(Conv2D(64,(3,3)),input_shape=())
vision_model.add(MaxPooling2D())

image_input = Input(shape=())
encoded_image = vision_model(image_input)

text_model = Sequential()
text_model.add(Dense(512, input_dim=512, activation='relu'))
text_model.add(Dropout(.5))
text_model.add(Dense(512, activation='relu'))
text_model.add(Dropout(.3))
text_model.add(Dense(16, activation='softmax'))

embedded_input = Input()
text_model(embedded_input)

merged = tf.layers.Concatenate([encoded_image, text_model])

In [ ]:
col1 = np.zeros((data.shape[0]),dtype=str).tolist()
max_images = 1
for p in range(data.shape[0]):
        try:
            url = imgs[p][0]
            filename = url.split('/')[-1]
            urllib.request.urlretrieve(url,"images/" + filename)
            col1[p] = filename
        except:
            pass